# Helsinki-NLP

In [10]:
from transformers import MarianMTModel, MarianTokenizer
import spacy
from datasets import load_dataset
import torch
import warnings
from tqdm import tqdm
import evaluate
import re

warnings.filterwarnings("ignore")

# 設定設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 載入 SpaCy NER 模型
nlp = spacy.load("en_core_web_trf")

# 載入 IWSLT 2017 英中翻譯資料集
dataset = load_dataset('iwslt2017', 'iwslt2017-en-zh', split='test[:10]', trust_remote_code=True)

# 使用 Helsinki-NLP 的英翻中模型
model_name = "Helsinki-NLP/opus-mt-en-zh"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)

# 實體識別和標記的函數，不替換文本，僅保存實體位置信息
def mark_entities(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
    return text, entities

# 後處理實體標記
def postprocess_translation(translated_text, entities):
    for ent_text, ent_label in entities:
        # 添加實體標記周圍的文本，根據需要修改格式
        entity_marker = f"r{ent_text}"  # 或使用 "《{}》".format(ent_text) 來加強顯示
        translated_text = translated_text.replace(ent_text, entity_marker)
    return translated_text

# 使用已訓練好的模型進行翻譯
def entity_aware_translate(input_text):
    # 先獲取實體位置信息，然後去標記翻譯
    clean_text, entities = mark_entities(input_text)
    inputs = tokenizer(clean_text, return_tensors="pt").to(device)
    translated_tokens = model.generate(
        inputs["input_ids"],
        max_length=256,
        num_beams=5,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    # 在翻譯文本中還原實體
    final_translation = postprocess_translation(translated_text, entities)
    return final_translation

# 計算 BLEU, METEOR 和 ROUGE 分數
meteor_metric = evaluate.load("meteor")
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

def evaluate_model(dataset):
    val_predictions = []
    val_references = []
    for example in tqdm(dataset, desc="Processing Validation Set"):
        input_text = example["translation"]["en"]
        reference_text = example["translation"]["zh"]
        
        # 進行翻譯
        final_translation = entity_aware_translate(input_text)
        val_predictions.append(final_translation)
        val_references.append([reference_text])
        
        # 顯示翻譯結果
        print("Original English Text:", input_text)
        print("Reference Text:", reference_text)
        print("Translated Text:", final_translation)
        print("=" * 50)
    
    # 計算 METEOR 和 BLEU 分數
    val_meteor_score = meteor_metric.compute(predictions=val_predictions, references=val_references)
    val_bleu_score = bleu_metric.compute(predictions=val_predictions, references=val_references)
    val_rouge_score = rouge_metric.compute(predictions=val_predictions, references=val_references)
    
    print("Validation Set METEOR score:", val_meteor_score)
    print("Validation Set BLEU score:", val_bleu_score)
    print("Validation Set ROUGE score:", val_rouge_score)

# 執行評估
evaluate_model(dataset)



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Processing Validation Set:  10%|█████▌                                                  | 1/10 [00:01<00:14,  1.63s/it]

Original English Text: Several years ago here at TED, Peter Skillman  introduced a design challenge  called the marshmallow challenge.
Reference Text: 几年前，在TED大会上， Peter Skillman 介绍了一个设计挑战 叫做“棉花糖挑战”
Translated Text: 几年前在rTED这里, rPeter Skillman介绍了一个设计挑战, 叫做棉花糖挑战。


Processing Validation Set:  20%|███████████▏                                            | 2/10 [00:02<00:09,  1.13s/it]

Original English Text: And the idea's pretty simple:  Teams of four have to build the tallest free-standing structure  out of 20 sticks of spaghetti,  one yard of tape, one yard of string  and a marshmallow.
Reference Text: 是个非常简单的主意 要求一组四人的团队搭建一个独立的最高建筑 材料是20根意大利面条 一段胶带，一段绳子 一块棉花糖
Translated Text: 这个想法很简单:四人小组必须从r20根意大利面条、一院带子、一院绳子和棉花糖中 建立最高标准的独立结构。


Processing Validation Set:  30%|████████████████▊                                       | 3/10 [00:02<00:05,  1.25it/s]

...

Processing Validation Set:  90%|██████████████████████████████████████████████████▍     | 9/10 [00:13<00:01,  1.78s/it]

Original English Text: They talk about it, they figure out what it's going to look like,  they jockey for power.
Reference Text: 他们讨论一阵， 勾勒出结果会是什么样子 他们互相争论一番对错
Translated Text: 他们谈论它,他们弄清楚 它会是什么样子, 他们骑的动力。


Processing Validation Set: 100%|███████████████████████████████████████████████████████| 10/10 [00:13<00:00,  1.39s/it]

Original English Text: Then they spend some time planning, organizing,  they sketch and they lay out spaghetti.
Reference Text: 然后花费点时间计划，组织 他们画一下图纸然后拿出意大利面条
Translated Text: 然后,他们花一些时间 规划,组织, 草图,他们放出意大利面条。


Validation Set METEOR score: {'meteor': 0.0}
Validation Set BLEU score: {'bleu': 0.0, 'precisions': [0.0, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.7058823529411764, 'translation_length': 58, 'reference_length': 34}
Validation Set ROUGE score: {'rouge1': 0.11333333333333333, 'rouge2': 0.06666666666666667, 'rougeL': 0.11333333333333333, 'rougeLsum': 0.11333333333333333}


# mt5

In [1]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import spacy
from datasets import load_dataset
import torch
import warnings
from tqdm import tqdm
import evaluate
import re

warnings.filterwarnings("ignore")

# 設定設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 載入 SpaCy NER 模型
nlp = spacy.load("en_core_web_trf")

# 載入 IWSLT 2017 英中翻譯資料集
dataset = load_dataset('iwslt2017', 'iwslt2017-en-zh', split='test[:10]', trust_remote_code=True)

# 使用 mT5-XXL 的模型和 Tokenizer
model_name = "google/mt5-large"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name).to(device)

# 實體識別和標記的函數，不替換文本，僅保存實體位置信息
def mark_entities(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
    return text, entities

# 後處理實體標記
def postprocess_translation(translated_text, entities):
    for ent_text, ent_label in entities:
        # 添加實體標記周圍的文本
        entity_marker = f"《{ent_text}》"
        translated_text = translated_text.replace(ent_text, entity_marker)
    return translated_text

# 使用預訓練模型進行翻譯

def entity_aware_translate(input_text):
    # 加入提示詞，以引導模型翻譯至中文
    prompt_text = f"translate English to Chinese: {input_text}"
    
    # 先獲取實體位置信息
    clean_text, entities = mark_entities(prompt_text)
    inputs = tokenizer(clean_text, return_tensors="pt").to(device)
    
    # 生成翻譯
    translated_tokens = model.generate(
        inputs["input_ids"],
        max_length=256,
        num_beams=5,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    
    # 在翻譯文本中還原實體
    final_translation = postprocess_translation(translated_text, entities)
    return final_translation

# 計算 BLEU, METEOR 和 ROUGE 分數
meteor_metric = evaluate.load("meteor")
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

def evaluate_model(dataset):
    val_predictions = []
    val_references = []
    for example in tqdm(dataset, desc="Processing Validation Set"):
        input_text = example["translation"]["en"]
        reference_text = example["translation"]["zh"]
        
        # 進行翻譯
        final_translation = entity_aware_translate(input_text)
        val_predictions.append(final_translation)
        val_references.append([reference_text])
        
        # 顯示翻譯結果
        print("Original English Text:", input_text)
        print("Reference Text:", reference_text)
        print("Translated Text:", final_translation)
        print("=" * 50)
    
    # 計算 METEOR, BLEU 和 ROUGE 分數
    val_meteor_score = meteor_metric.compute(predictions=val_predictions, references=val_references)
    val_bleu_score = bleu_metric.compute(predictions=val_predictions, references=val_references)
    val_rouge_score = rouge_metric.compute(predictions=val_predictions, references=val_references)
    
    print("Validation Set METEOR score:", val_meteor_score)
    print("Validation Set BLEU score:", val_bleu_score)
    print("Validation Set ROUGE score:", val_rouge_score)

# 執行評估
evaluate_model(dataset)


C:\Users\USER\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Processing Validation Set:  10%|█████▌                               

Original English Text: Several years ago here at TED, Peter Skillman  introduced a design challenge  called the marshmallow challenge.
Reference Text: 几年前，在TED大会上， Peter Skillman 介绍了一个设计挑战 叫做“棉花糖挑战”
Translated Text: <extra_id_0> translate 《English》: The marshmallow challenge <extra_id_1> translate 《English》: The marshmallow challenge at 《TED》 <extra_id_2> translate 《English》: The marshmallow challenge at 《TED》 <extra_id_3> translate 《English》: The marshmallow challenge at 《TED》 <extra_id_4> translate 《English》: The marshmallow challenge at 《TED》 <extra_id_5>  <extra_id_6> 《English》: The marshmallow challenge at 《TED》 <extra_id_7>  <extra_id_8> 《English》: The marshmallow challenge <extra_id_9> 《English》: The marshmallow challenge <extra_id_10> 《English》: <extra_id_11> 《English》 <extra_id_12> 《English》 <extra_id_13> 《English》 <extra_id_14> 《English》: <extra_id_15> 《English》 <extra_id_16> 《English》: <extra_id_17> 《English》: <extra_id_18> 《English》: <extra_id_19> 《English》: <extra_id_20> 《

Processing Validation Set:  20%|███████████▏                                            | 2/10 [00:27<01:52, 14.11s/it]

...

Processing Validation Set:  90%|██████████████████████████████████████████████████▍     | 9/10 [01:42<00:10, 10.02s/it]

Original English Text: They talk about it, they figure out what it's going to look like,  they jockey for power.
Reference Text: 他们讨论一阵， 勾勒出结果会是什么样子 他们互相争论一番对错
Translated Text: <extra_id_0> 《English》 to 《Chinese》: They talk about it, they talk about it, they jockey for power. They talk about it, they talk about it, they jockey for power <extra_id_1> 《English》 to 《Chinese》: They translate 《English》 to 《Chinese》: They translate 《English》 to 《Chinese》: <extra_id_2> 《English》 to 《Chinese》: They translate 《English》 to 《Chinese》: They <extra_id_3> 《English》 to 《Chinese》: They translate <extra_id_4> 《English》 to 《Chinese》: They translate 《English》 to 《Chinese》: They <extra_id_5> 《English》 <extra_id_25> 《Chinese》: <extra_id_26> 《Chinese》 <extra_id_55>


Processing Validation Set: 100%|███████████████████████████████████████████████████████| 10/10 [01:54<00:00, 11.40s/it]

Original English Text: Then they spend some time planning, organizing,  they sketch and they lay out spaghetti.
Reference Text: 然后花费点时间计划，组织 他们画一下图纸然后拿出意大利面条
Translated Text: <extra_id_0> translate 《English》: Then they spend some time planning, organizing, organizing and they sketch and they lay out spaghetti. Then they spend some time planning, organizing.  <extra_id_15> translate 《English》:  <extra_id_16>:  <extra_id_17>:  <extra_id_18>:  <extra_id_19>:  <extra_id_20>:  <extra_id_21>:  <extra_id_22>:  <extra_id_23>:  <extra_id_18>:  <extra_id_19>:  <extra_id_20>:  <extra_id_21>:  <extra_id_22>:  <extra_id_23>:  <extra_id_24>:  <extra_id_25>:  <extra_id_26>:  <extra_id_27>:  <extra_id_28>:  <extra_id_29>:  <extra_id_30>: Then they spend some time planning, organizing, they sketch and spaghetti.  <extra_id_31>:  <extra_id_32>:  <extra_id_33>:  <extra_id_34>:  <extra_id_35>:  <extra_id_36>:  <extra_id_37>:  <extra_id_38>:


Validation Set METEOR score: {'meteor': 0.0}
Validation Set BLEU score: {'bleu': 0.0, 'precisions': [0.0, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 73.29411764705883, 'translation_length': 2492, 'reference_length': 34}
Validation Set ROUGE score: {'rouge1': 0.002749719416386083, 'rouge2': 0.0, 'rougeL': 0.002749719416386083, 'rougeLsum': 0.002749719416386083}
